Дане домашнє завдання буде повністю пов'язане з лінійною регресією та її реалізацією. Отож розіб'ємо наше домашнє завдання на декілька частин:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Files/Housing.csv')

In [4]:
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


# 1. напишіть функцію гіпотези лінійної регресії у векторному вигляді;

In [5]:
import numpy as np

def hypothesis_function(X, weight):
  """
  Ця функція обчислює гіпотезу лінійної регресії у векторному вигляді.

  Аргументи:
    X: Матриця незалежних змінних з розміром (n_samples, n_features).
    weight: Вектор параметрів з розміром (n_features, ).

  Повертає:
    h_weight: Вектор прогнозованих значень з розміром (n_samples, ).
  """

  # Перевірка розмірів
  if X.shape[1] != weight.shape[0]:
    raise ValueError("Розмірності X та weight не співпадають!")

  # Обчислення гіпотези
  h_weight = np.dot(X, weight)

  return h_weight

Приклад використання:

In [6]:
import numpy as np

# Припустимо, що у нас є наступні дані:
X = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])
beta = np.array([
    [0.5],
    [1.0],
    [1.5]
])

# Обчислимо прогнозовані значення y:
h = hypothesis_function(X, beta)
print(h)

[[ 7.]
 [16.]
 [25.]]


# 2. створіть функцію для обчислення функції втрат у векторному вигляді

In [7]:
import numpy as np

def loss_function(X, y, h, beta):
  """
  Ця функція обчислює значення функції втрат MSE (Mean Squared Error) для заданих значень X, y та beta.

  Args:
    X: матриця незалежних змінних розмірності (n, k), де n - кількість точок даних,
         a k - кількість незалежних змінних.
    y: вектор цільових значень розмірності (n, 1).
    beta: вектор коефіцієнтів моделі розмірності (k, 1).

  Returns:
    loss: скалярне значення, яке представляє сумарну помилку моделі.
  """

  #h = hypothesis_function(X, beta)
  diff = h - y
  squared_error = np.dot(diff.T, diff)
  loss = squared_error / (2 * len(y))
  return loss


In [8]:
import numpy as np

# Припустимо, що у нас є наступні дані:
X = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])
y = np.array([
    [4],
    [8],
    [12]
])
beta = np.array([
    [0.5],
    [1.0],
    [1.5]
])

# Обчислимо значення функції втрат:
h = hypothesis_function(X, beta)
loss = loss_function(X, y, h, beta)
print(loss)


[[40.33333333]]


# 3. реалізуйте один крок градієнтного спуску;

In [9]:
import numpy as np

def compute_gradient(X, y, h, theta):
    """
    Обчислює градієнт функції втрат MSE(Mean Squared Error)для лінійної регресії.

    Параметри:
    X : np.array
        Матриця ознак (з включеним стовпцем одиниць для вільного члена).
    y : np.array
        Вектор цільових значень.
    theta : np.array
        Вектор параметрів моделі (включаючи вільний член).

    Повертає:
    grad : np.array
        Градієнт функції втрат по параметрах theta.
    """
    m = len(y)  # кількість прикладів
    #h = hypothesis_function(X, theta) # h_weight = np.dot(X, weight)
    diff = h - y            # (h(X, β) - y)
    grad  = np.dot(X.T, diff) # X^T * (h(X, β) - y)
    grad = grad  / m # ∇E(β) = (1/N) * X^T * (h(X, β) - y)

    return grad

def gradient_descent_step(X, y, h, theta, grad_dict, learning_rate=0.01):
    """
    Виконує один крок градієнтного спуску.

    Параметри:
    X : np.array
        Матриця ознак.
    y : np.array
        Вектор цільових значень.
    theta : np.array
        Вектор параметрів моделі.
    learning_rate : float
        Швидкість навчання.

    Повертає:
    theta : np.array
        Оновлений вектор параметрів моделі.
    """
    #h = hypothesis_function(X, theta)


    grad = compute_gradient(X, y, h, theta)
    theta = theta - learning_rate * grad
    grad_dict ['value']= grad
    return theta


In [10]:
import numpy as np

X = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([3, 5, 7])
theta = np.array([0.5, 0.2])
learning_rate = 0.1

h = hypothesis_function(X, theta)
grad_dict = {'value': 0.}
theta_updated = gradient_descent_step(X, y, h, theta, grad_dict, learning_rate)
print(f"theta_updated: {theta_updated}, grad: {grad_dict['value']}")

theta_updated: [1.39 1.36], grad: [ -8.9 -11.6]


Градієнтний спуск (всі кроки):

In [11]:
def gradient_descent(X, y, iterations=1000, learning_rate=0.01, stopping_threshold=0.0000001):
    """
    Виконує градієнтний спуск для вказаної кількості ітерацій.

    Параметри:
    X : np.array
        Матриця ознак.
    y : np.array
        Вектор цільових значень.
    theta : np.array
        Початковий вектор параметрів моделі.
    learning_rate : float
        Швидкість навчання.
    iterations : int
        Кількість ітерацій.

    Повертає:
    theta : np.array
        Оптимізований вектор параметрів моделі.
    """

    m, n = X.shape
    X = np.hstack((np.ones((m, 1)), X))  # Додаємо колонку одиниць для інтерцепта
    theta = np.zeros(n + 1)  # Ініціалізація ваг (інтерцепт + коефіцієнти)

    grad_dict = {'value': 0.}
    previous_cost = None

    for i in range(iterations):

        h = hypothesis_function(X, theta)
        cost = loss_function(X, y, h, theta)
        if cost == np.Inf:
          break

        if previous_cost and abs(previous_cost - cost) <= stopping_threshold:
            break

        previous_cost = cost

        theta = gradient_descent_step(X, y, h, theta, grad_dict, learning_rate)
        print(f"iteration: {i}: theta: {theta} cost: {cost} grad: {grad_dict['value']}")
    return theta


In [48]:
from tabulate import tabulate

def print_analitycal_errors(X2,theta2,y,isNeedToAddOnesToX=True):
  # print(X.shape)
  # print(theta.shape)

  # X = np.hstack((np.ones((m, 1)), X))
  # print(X.shape)
  # print(theta.shape)
  m, n = X2.shape
  if isNeedToAddOnesToX:
    X2 = np.hstack((np.ones((m, 1)), X2))
  y_pred = hypothesis_function(X2,theta2)

  # Оцінка моделі
  mse = mean_squared_error(y, y_pred)
  rmse = mean_squared_error(y, y_pred, squared=False)  # або np.sqrt(mse)
  mae = mean_absolute_error(y, y_pred)
  r2 = r2_score(y, y_pred)

  # print(f"Mean Squared Error (MSE): {mse}")
  # print(f"Root Mean Squared Error (RMSE): {rmse}")
  # print(f"Mean Absolute Error (MAE): {mae}")
  # print(f"R^2 Score: {r2}")

  # Виведення показників у таблицю
  metrics_table = [
      ["Metric", "Values"],
      ["MSE", mse],
      ["RMSE", rmse],
      ["MAE", mae],
      ["R²", r2]
  ]

  print(tabulate(metrics_table, headers="firstrow", tablefmt="grid"))

  errors = {"mse":mse, "rmse": rmse, "mae": mae, "r2": r2}
  return errors

Приклад використання:

In [67]:
# # Приклад використання
# X = np.array([[1, 1], [1, 2], [1, 3]])  # Матриця ознак з додаванням стовпця одиниць
# y = np.array([1, 2, 3])  # Вектор цільових значень

#theta = np.array([0.1, 0.1])

# Генерація випадкової матриці X (20 зразків, 1 ознака) і вектора Y
# Для простоти візуалізації використовуємо одну ознаку
X = np.random.rand(20, 1) * 10  # Масштабування для кращої візуалізації
y = 2.5 * X.flatten() + np.random.randn(20) * 2  # Істинні коефіцієнти з шумом
#flatten - Return a copy of the array collapsed into one dimension.
#random.randn(d0, d1, ..., dn) - Return a sample (or samples) from the “standard normal” distribution.


learning_rate=0.001
iterations=1000
stopping_threshold=0.01

# Виконання градієнтного спуску
theta = gradient_descent(X, y, iterations, learning_rate, stopping_threshold)
print("Результат оптимізації:", theta)



iteration: 0: theta: [0.01327602 0.08510405] cost: 107.66141060910032 grad: [-13.27602468 -85.10405482]
iteration: 1: theta: [0.02609267 0.16723523] cost: 100.37200917965768 grad: [-12.81664401 -82.13117241]
iteration: 2: theta: [0.03846597 0.24649734] cost: 93.58287383888882 grad: [-12.37330618 -79.26211673]
iteration: 3: theta: [0.05041143 0.32299061] cost: 87.25967163238877 grad: [-11.94545093 -76.49326167]
iteration: 4: theta: [0.06194396 0.39681171] cost: 81.37042585535396 grad: [-11.53253751 -73.82110777]
iteration: 5: theta: [0.07307801 0.46805399] cost: 75.885354344617 grad: [-11.1340441  -71.24227778]
iteration: 6: theta: [0.08382747 0.5368075 ] cost: 70.7767188686006 grad: [-10.74946708 -68.75351241]
iteration: 7: theta: [0.09420579 0.60315917] cost: 66.018684853546 grad: [-10.37832043 -66.35166619]
iteration: 8: theta: [0.10422593 0.66719287] cost: 61.587190736644835 grad: [-10.0201351 -64.0337035]
iteration: 9: theta: [0.11390039 0.72898957] cost: 57.459826285384864 grad: [

In [68]:
# print(X.shape)
# print(theta.shape)

# X = np.hstack((np.ones((m, 1)), X))
# print(X.shape)
# print(theta.shape)
print_analitycal_errors(X,theta,y)

+----------+----------+
| Metric   |   Values |
+==========+==========+
| MSE      | 3.15832  |
+----------+----------+
| RMSE     | 1.77717  |
+----------+----------+
| MAE      | 1.32053  |
+----------+----------+
| R²       | 0.919163 |
+----------+----------+


{'mse': 3.1583178853404617,
 'rmse': 1.7771656887697505,
 'mae': 1.3205297766351605,
 'r2': 0.9191625620978223}

# 4. знайдіть найкращі параметри
𝑤
⃗
w
  для датасету використовуючи написані вами функції, прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень;

In [82]:
df = pd.read_csv('/content/drive/MyDrive/Files/Housing.csv')
X = df[['area', 'bedrooms', 'bathrooms']]
y = df['price']
y

0      13300000
1      12250000
2      12250000
3      12215000
4      11410000
         ...   
540     1820000
541     1767150
542     1750000
543     1750000
544     1750000
Name: price, Length: 545, dtype: int64

In [105]:
df = pd.read_csv('/content/drive/MyDrive/Files/Housing.csv')
df_copy = df.copy()

X = df[['area', 'bedrooms', 'bathrooms']]
y = df['price']
#y = y/1000

# Стандартизація (є від'ємні значення, зберігає розподіл)
from sklearn.preprocessing import StandardScaler

# Створіть об'єкт StandardScaler
scaler = StandardScaler()

# Масштабуйте значення стовпців "area", "bedrooms", "bathrooms"
X_scaled = scaler.fit_transform(X)

# # Оновіть df з масштабованими значеннями
# df['area'] = X_scaled[:, 0]
# df['bedrooms'] = X_scaled[:, 1]
# df['bathrooms'] = X_scaled[:, 2]

# Нормалізація (щоб уникнути від'ємних значень) вже робимо Стандартизацію
# from sklearn.preprocessing import MinMaxScaler

# # Створіть об'єкт MinMaxScaler
# scaler = MinMaxScaler()

# # Масштабуйте значення стовпців "area", "bedrooms", "bathrooms"
# X_scaled = scaler.fit_transform(X)

# # Оновіть df з масштабованими значеннями
# df['area'] = X_scaled[:, 0]
# df['bedrooms'] = X_scaled[:, 1]
# df['bathrooms'] = X_scaled[:, 2]

learning_rate=0.00001
iterations=1000
stopping_threshold=1

# Виконання градієнтного спуску
theta = gradient_descent(X, y, iterations, learning_rate, stopping_threshold)
print("Результат оптимізації:", theta)



iteration: 0: theta: [4.76672925e+01 2.67229201e+05 1.46390263e+02 6.61666767e+01] cost: 13106916364659.266 grad: [-4.76672925e+06 -2.67229201e+10 -1.46390263e+07 -6.61666767e+06]
iteration: 1: theta: [-1.36684214e+04 -8.29184347e+07 -4.11674456e+04 -1.81348839e+04] cost: 1.1079248519412027e+18 grad: [1.37160887e+09 8.31856639e+12 4.13138358e+09 1.82010505e+09]
iteration: 2: theta: [4.25712905e+06 2.58119106e+10 1.28236543e+07 5.65005810e+06] cost: 1.0735907687423356e+23 grad: [-4.27079747e+11 -2.58948290e+15 -1.28648217e+12 -5.66819298e+11]
iteration: 3: theta: [-1.32519643e+09 -8.03497810e+12 -3.99185683e+09 -1.75879686e+09] cost: 1.0403222542597307e+28 grad: [1.32945356e+14 8.06079001e+17 4.00468049e+14 1.76444692e+14]
iteration: 4: theta: [4.12519822e+11 2.50120482e+15 1.24262339e+12 5.47495118e+11] cost: 1.0080846671145904e+33 grad: [-4.13845018e+16 -2.50923980e+20 -1.24661524e+17 -5.49253914e+16]
iteration: 5: theta: [-1.28413115e+14 -7.78598957e+17 -3.86815692e+14 -1.70429516e+1

In [106]:
def print_coefficients(theta):
  # Виведення коефіцієнтів
  coefficients = pd.DataFrame({
      'Feature': ['Intercept'] + list(['area', 'bedrooms', 'bathrooms']),
      'Coefficient': theta
  })
  print("\nКоефіцієнти моделі:")
  print(coefficients)

In [107]:
print_coefficients(theta)


Коефіцієнти моделі:
     Feature    Coefficient
0  Intercept  1.763320e+146
1       area  1.069142e+150
2   bedrooms  5.311605e+146
3  bathrooms  2.340273e+146


In [108]:
print_analitycal_errors(X,theta,y)

+----------+-----------------+
| Metric   |          Values |
+==========+=================+
| MSE      |  inf            |
+----------+-----------------+
| RMSE     |  inf            |
+----------+-----------------+
| MAE      |    5.50666e+153 |
+----------+-----------------+
| R²       | -inf            |
+----------+-----------------+


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:446: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:446: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:927: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
/usr/local/lib/python3.10/dist-

{'mse': inf, 'rmse': inf, 'mae': 5.5066635289466255e+153, 'r2': -inf}

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Поділ на навчальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабування ознак
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


learning_rate=0.001
iterations=1000
stopping_threshold=0.01

# Виконання градієнтного спуску
theta = gradient_descent(X_test_scaled, y_test, iterations, learning_rate, stopping_threshold)
print("Результат оптимізації:", theta)



iteration: 0: theta: [5007.53669725 1143.1813621   976.70980891 2359.48398578] cost: 15064997010321.102 grad: [-5007536.69724771 -1143181.36210248  -976709.80890567 -2359483.9857837 ]
iteration: 1: theta: [10009.53387207  2284.82768817  1951.3204196   4713.64201912] cost: 15032115651686.885 grad: [-5001997.17481986 -1141646.32607034  -974610.61069711 -2354158.03333819]
iteration: 2: theta: [15005.99826883  3424.9415908   2923.83610585  7062.48464532] cost: 14999322383462.643 grad: [-4996464.39676037 -1140113.90262817  -972515.68625067 -2348842.62620066]
iteration: 3: theta: [19996.93662276  4563.52567772  3894.26113309  9406.02238975] cost: 14966616943026.688 grad: [-4990938.35393077 -1138584.08691658  -970425.02723437 -2343537.74442949]
iteration: 4: theta: [24982.35565996  5700.5825518   4862.59975842 11744.26575787] cost: 14933999068606.473 grad: [-4985419.03720625 -1137056.87408534  -968338.62533206 -2338243.3681201 ]
iteration: 5: theta: [29962.26209744  6836.1148111   5828.856230

In [103]:
print_coefficients(theta)


Коефіцієнти моделі:
     Feature   Coefficient
0  Intercept  3.071318e+06
1       area  6.664913e+05
2   bedrooms  3.918727e+05
3  bathrooms  8.574034e+05


In [104]:
print_analitycal_errors(X_test_scaled,theta,y_test)

+----------+--------------+
| Metric   |       Values |
+==========+==============+
| MSE      |  5.85789e+12 |
+----------+--------------+
| RMSE     |  2.42031e+06 |
+----------+--------------+
| MAE      |  1.91222e+06 |
+----------+--------------+
| R²       | -0.158929    |
+----------+--------------+


{'mse': 5857886429482.215,
 'rmse': 2420307.0940445173,
 'mae': 1912218.3430338535,
 'r2': -0.15892868116750858}

# знайдіть ці ж параметри за допомогою аналітичного рішення;

In [56]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Files/Housing.csv')

# Відокремлення цільової змінної та ознак
X = df[['area', 'bedrooms', 'bathrooms']] #df.drop('price', axis=1)
y = df['price']

# Додавання стовпчика одиниць для інтерцепта
X = np.hstack((np.ones((X.shape[0], 1)), X))

# Конвертація в numpy масиви
X = np.array(X)
y = np.array(y)

# Аналітичне рішення для знаходження коефіцієнтів
# theta = (X^T * X)^(-1) * X^T * y
theta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)



In [57]:
print_coefficients(theta)


Коефіцієнти моделі:
     Feature   Coefficient
0  Intercept -1.731716e+05
1       area  3.787628e+02
2   bedrooms  4.068200e+05
3  bathrooms  1.386049e+06


In [58]:
print_analitycal_errors(X,theta,y,False)

+----------+------------------+
| Metric   |           Values |
+==========+==================+
| MSE      |      1.79117e+12 |
+----------+------------------+
| RMSE     |      1.33835e+06 |
+----------+------------------+
| MAE      | 999110           |
+----------+------------------+
| R²       |      0.487083    |
+----------+------------------+


{'mse': 1791170049977.3193,
 'rmse': 1338346.0127998735,
 'mae': 999109.7245946018,
 'r2': 0.4870830667058762}

# SGDRegressor

In [59]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

df = pd.read_csv('/content/drive/MyDrive/Files/Housing.csv')

# Відокремлення цільової змінної та ознак
X = df[['area', 'bedrooms', 'bathrooms']] #df.drop('price', axis=1)
y = df['price']

# Поділ на навчальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабування ознак
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Створення та навчання моделі SGDRegressor
model = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)
model.fit(X_train_scaled, y_train)

# Передбачення на тестовій вибірці
y_pred = model.predict(X_test_scaled)

# Оцінка моделі
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Оцінка моделі
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)  # або np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R^2 Score: {r2}")


# # Виведення коефіцієнтів моделі
# print(f"Model coefficients: {model.coef_}")
# print(f"Model intercept: {model.intercept_}")

# Виведення коефіцієнтів моделі у табличку
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
})
print("\nКоефіцієнти моделі:")
print(coefficients)

# Додавання інтерцепта до таблички
intercept = pd.DataFrame({
    'Feature': ['Intercept'],
    'Coefficient': [model.intercept_[0]]
})
coefficients = pd.concat([intercept, coefficients], ignore_index=True)
print("\nКоефіцієнти моделі з інтерцептом:")
print(coefficients)


Mean Squared Error: 2752935114161.8574
Mean Squared Error (MSE): 2752935114161.8574
Root Mean Squared Error (RMSE): 1659197.1293857333
Mean Absolute Error (MAE): 1266167.5167849048
R^2 Score: 0.45535723513893156

Коефіцієнти моделі:
     Feature    Coefficient
0       area  759726.597854
1   bedrooms  269588.174858
2  bathrooms  671353.656875

Коефіцієнти моделі з інтерцептом:
     Feature   Coefficient
0  Intercept  4.701252e+06
1       area  7.597266e+05
2   bedrooms  2.695882e+05
3  bathrooms  6.713537e+05


# для перевірки спрогнозованих значень, використайте LinearRegression з бібліотеки scikit-learn та порівняйте результати.

In [60]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tabulate import tabulate

# Приклад DataFrame
df = pd.read_csv('/content/drive/MyDrive/Files/Housing.csv')

# Відокремлення цільової змінної та ознак
X = df[['area', 'bedrooms', 'bathrooms']]
y = df['price']

# Аналітичне рішення
X_b = np.hstack((np.ones((X.shape[0], 1)), X))
theta_analytical = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
y_pred_analytical = X_b.dot(theta_analytical)

# Використання LinearRegression з scikit-learn
model = LinearRegression()
model.fit(X, y)
y_pred_sklearn = model.predict(X)

# Оцінка аналітичного рішення
mse_analytical = mean_squared_error(y, y_pred_analytical)
mae_analytical = mean_absolute_error(y, y_pred_analytical)
r2_analytical = r2_score(y, y_pred_analytical)

# Оцінка моделі scikit-learn
mse_sklearn = mean_squared_error(y, y_pred_sklearn)
mae_sklearn = mean_absolute_error(y, y_pred_sklearn)
r2_sklearn = r2_score(y, y_pred_sklearn)

# # Порівняння результатів
# print("Аналітичне рішення:")
# print(f"Mean Squared Error (MSE): {mse_analytical}")
# print(f"Mean Absolute Error (MAE): {mae_analytical}")
# print(f"R^2 Score: {r2_analytical}")

# print("\nРішення scikit-learn:")
# print(f"Mean Squared Error (MSE): {mse_sklearn}")
# print(f"Mean Absolute Error (MAE): {mae_sklearn}")
# print(f"R^2 Score: {r2_sklearn}")

# Виведення показників у таблицю
metrics_table = [
    ["Metric", "Analytical Solution", "scikit-learn Solution"],
    ["MSE", mse_analytical, mse_sklearn],
    ["MAE", mae_analytical, mae_sklearn],
    ["R²", r2_analytical, r2_sklearn]
]

print(tabulate(metrics_table, headers="firstrow", tablefmt="grid"))


# Виведення коефіцієнтів моделі
coefficients_analytical = pd.DataFrame({
    'Feature': ['Intercept'] + list(X.columns),
    'Coefficient': theta_analytical
})
print("\nКоефіцієнти моделі (аналітичне рішення):")
print(coefficients_analytical)

coefficients_sklearn = pd.DataFrame({
    'Feature': ['Intercept'] + list(X.columns),
    'Coefficient': [model.intercept_] + list(model.coef_)
})
print("\nКоефіцієнти моделі (scikit-learn):")
print(coefficients_sklearn)


+----------+-----------------------+-------------------------+
| Metric   |   Analytical Solution |   scikit-learn Solution |
+==========+=======================+=========================+
| MSE      |           1.79117e+12 |             1.79117e+12 |
+----------+-----------------------+-------------------------+
| MAE      |      999110           |        999110           |
+----------+-----------------------+-------------------------+
| R²       |           0.487083    |             0.487083    |
+----------+-----------------------+-------------------------+

Коефіцієнти моделі (аналітичне рішення):
     Feature   Coefficient
0  Intercept -1.731716e+05
1       area  3.787628e+02
2   bedrooms  4.068200e+05
3  bathrooms  1.386049e+06

Коефіцієнти моделі (scikit-learn):
     Feature   Coefficient
0  Intercept -1.731716e+05
1       area  3.787628e+02
2   bedrooms  4.068200e+05
3  bathrooms  1.386049e+06
